# Bag of Visual words

This project come from HJP_Computer_vision class 3rd assignmentin SKKU(SungKyunKwanUniv).
 


- Load_data, extract_patches
- extract features
- Normalize
- Codebook
- encode features
- Approximate kernel
- classify images with SVM
- print res

In [1]:
import time
import cv2
import glob
import os
import numpy as np
import random
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.kernel_approximation import AdditiveChi2Sampler
from sklearn.svm import LinearSVC

print(cv2.__version__)

4.5.2


In [2]:
'''
Load Dataset
'''
def scene15():

    train_folders = glob.glob("SCENE-15/train/*")
    train_folders.sort()
    classes = dict()
    x_train = list()
    y_train = list()
    for index, folder in enumerate(train_folders):
        label = os.path.basename(folder) #return the base name of pathname  "folder"
        classes[label] = index
        paths = glob.glob(os.path.join(folder, "*"))
        for path in paths:
            x_train.append(cv2.imread(path, 0))
            y_train.append(index)

    x_test = list()
    y_test = list()
    test_folders = glob.glob("SCENE-15/test/*")
    test_folders.sort()
    for folder in test_folders:
        label = os.path.basename(folder)
        index = classes[label]
        paths = glob.glob(os.path.join(folder, "*"))
        for path in paths:
            x_test.append(cv2.imread(path, 0))
            y_test.append(index)

    return x_train, y_train, x_test, y_test, sorted(classes.keys())

In [3]:
x_train, y_train, x_test, y_test, labels_names = scene15()

random_indices = list(range(len(y_train)))
random.shuffle(random_indices)
x_train = np.array(x_train)[random_indices].tolist() #여기다가 dtype=object?
y_train = np.array(y_train)[random_indices].tolist()

<ipython-input-3-48c5413a842b>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train = np.array(x_train)[random_indices].tolist() #여기다가 dtype=object?


In [167]:
'''
Parameters
'''
patch_stride = 32

In [168]:
'''
Extract Patches
'''
train_key_points = list()
train_feature_shapes = list()
for image in x_train:
    h, w = image.shape
    image_key_points = list()
    for x in range(0, w, patch_stride):
        for y in range(0, h, patch_stride):
            image_key_points.append(cv2.KeyPoint(x, y, patch_stride))
    train_key_points.append(image_key_points)
    train_feature_shapes.append((len(range(0, w, patch_stride)), (len(range(0, h, patch_stride)))))

In [169]:
test_key_points = list()
test_feature_shapes = list()

for image in x_test:
    h, w = image.shape
    image_key_points = list()
    for x in range(0, w, patch_stride):
        for y in range(0, h, patch_stride):
            image_key_points.append(cv2.KeyPoint(x, y, patch_stride))
    test_key_points.append(image_key_points)
    test_feature_shapes.append((len(range(0, w, patch_stride)), (len(range(0, h, patch_stride)))))

[문제 1. 특징 추출 – 15 pts]  

1-1. 주어진 Keypoint를 통하여, 각각의 이미지마다 특징을 추출하시오.  
이 때, OpenCV 패키지를 활용하여, SIFT와 같은 특징을 추출하시오. (자세한 사항은 템플릿 코드를 참고)  

'''
Extract Features
'''  
#1) descriptor를 선정하세요. (SIFT, SURF 등) OpenCV의 패키지를 사용하시면 됩니다.  
#2) for 반복문 안에서, 1)에서 정의한 descriptor를 통하여 features를 추출하세요. features의 차원은 (# of keypoints, feature_dim) 입니다.  

In [183]:
descriptor = cv2.SIFT_create()
#descriptor = cv2.xfeatures2d.SURF_create(1000, 3, True, True)
#descriptor = cv2.ORB_create()

In [184]:
train_features = []
index = 0

for image, key_points in zip(x_train, train_key_points):
    _, features = descriptor.detectAndCompute(image, None)
    train_features.append(features)
    index += 1
    #print("Extract Train Features ... {:4d}/{:4d}".format(index, len(x_train)))
print("train feature extract finished")

train feature extract finished


In [185]:
test_features = []
index = 0

for image, key_points in zip(x_test, test_key_points):
    _, features = descriptor.detectAndCompute(image, None)
    test_features.append(features)
    index += 1
    #print("Extract Test Features ... {:4d}/{:4d}".format(index, len(x_test)))
print("test feature extract finished")

test feature extract finished


In [186]:
'''
Normalizing
'''
flattened_train_features = np.concatenate(train_features, axis=0)
pca = PCA(n_components=flattened_train_features.shape[-1], whiten=True)
pca.fit(flattened_train_features)

PCA(n_components=128, whiten=True)

In [187]:
train_normalized_features = list()
index = 0

for features in train_features:
    features = pca.transform(features)
    train_normalized_features.append(features)
    index += 1
    #print("Normalize Train Features ... {:4d}/{:4d}".format(index, len(train_features)))

print("train features nomalized")

train features nomalized


In [188]:
test_normalized_features = list()
index = 0

for features in test_features:
    features = pca.transform(features)
    test_normalized_features.append(features)
    index += 1
    #print("Normalize Test Features ... {:4d}/{:4d}".format(index, len(test_features)))

print("test features nomalized")

test features nomalized


[문제 2. Bag-of-Features 구현 – 30 pts]  

2-1.   
K-means을 통해 구해진 codebook을 통하여, 각각의 이미지의 특징을 인코딩(histogram화 혹은  양자화 라고도 함) 하시오.  
(자세한 사항은 템플릿 코드를 참고)  

'''
Make Codebook
'''

In [190]:
#import sklearn.preprocessing import StandardScaler

class Codebook:

    def __init__(self, K):

        self.K = K
        
        self.kmeans = KMeans(n_clusters=K, verbose=True) #유사한 피쳐끼리 클러스터링 한다.

    def make_code_words(self, features):

        self.kmeans.fit(features)

    def encode(self, features, shapes):

        distances = self.kmeans.transform(features) #centroids, codewords와 각 이미지 특징들 간의 거리

        # reshaped_features는 Spatial Pyramid Matching 문제에 활용하세요.
        #reshaped_features = np.reshape(features, (shapes[0], shapes[1], -1))
        
        
        # 1) 함수 encode 부분 안의 None 부분을 채우세요.
        #    distances는 K means 알고리즘을 통해 얻어진 centroids, 즉 codewords(visual words)와 각 이미지의 특징들 간의 거리 입니다.
        #    distances 값을 이용하여, features(# of keypoints, feature_dim)를 인코딩(histogram 혹은 quantization이라고도 함) 하세요.
        #    인코딩된 결과인 representations은 (K)로 표현되어야 합니다. 이 때, K는 codewords의 개수입니다.
        
        #bins =[0, 20, 40, 60, 80, 100] #k==5
        #bins = [0, 10, 20, 30, 40, 50, 60, 70,80, 90, 100] #k==10
        bins =[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100] #k == 20
        
        num_of_codewords, _ = np.histogram(distances, bins = bins)
        
        if np.array(num_of_codewords).shape != (self.K, ):
            # representations는 반드시 (K) 차원을 가져야 합니다 (Spatial Pyramid Matching 사용 안할 시에만).
            print(np.array(num_of_codewords).shape)
            print(self.K)
            print("Your code may be wrong")

        return num_of_codewords

Kmeans clustering을 통해서 뽑힌 무작위 feature들로부터 공통점들을 찾아 군집을 만들고  
  
  
그 거리에 따라서, feature들( 얘내가 어떤 애인지는 우리의 관심사가 아님 )을 묶은 애들을   

histogram에 넣는다. 다만 히스토그램을 normalize 해야한다.

이 히스토그램이 사실상 bag of visual words를 담당하므로 사실상 이 코드가 핵심이다. 
  
vector화 된 히스토그램 자료들을 ML이 이제 구분해낸다.

In [191]:
K = 20

In [192]:
flattened_normalized_train_features = pca.transform(flattened_train_features)
codebook = Codebook(K)
codebook.make_code_words(flattened_normalized_train_features)

Initialization complete
Iteration 0, inertia 119236400.0
Iteration 1, inertia 96769768.0
Iteration 2, inertia 94905584.0
Iteration 3, inertia 94017216.0
Iteration 4, inertia 93582712.0
Iteration 5, inertia 93373984.0
Iteration 6, inertia 93264840.0
Iteration 7, inertia 93186832.0
Iteration 8, inertia 93126800.0
Iteration 9, inertia 93069120.0
Iteration 10, inertia 93010880.0
Iteration 11, inertia 92959888.0
Iteration 12, inertia 92922648.0
Iteration 13, inertia 92896096.0
Iteration 14, inertia 92876592.0
Iteration 15, inertia 92857672.0
Iteration 16, inertia 92840752.0
Iteration 17, inertia 92823168.0
Iteration 18, inertia 92809592.0
Iteration 19, inertia 92798112.0
Iteration 20, inertia 92789936.0
Iteration 21, inertia 92781984.0
Iteration 22, inertia 92773592.0
Iteration 23, inertia 92761424.0
Iteration 24, inertia 92744368.0
Iteration 25, inertia 92731624.0
Iteration 26, inertia 92724008.0
Iteration 27, inertia 92720256.0
Iteration 28, inertia 92718584.0
Iteration 29, inertia 927169

Iteration 34, inertia 92589104.0
Iteration 35, inertia 92588560.0
Iteration 36, inertia 92587928.0
Iteration 37, inertia 92587936.0
Iteration 38, inertia 92587616.0
Iteration 39, inertia 92587272.0
Iteration 40, inertia 92587184.0
Iteration 41, inertia 92586952.0
Iteration 42, inertia 92586248.0
Iteration 43, inertia 92585800.0
Iteration 44, inertia 92585088.0
Iteration 45, inertia 92584448.0
Iteration 46, inertia 92583480.0
Iteration 47, inertia 92582392.0
Iteration 48, inertia 92580448.0
Iteration 49, inertia 92579032.0
Iteration 50, inertia 92577728.0
Iteration 51, inertia 92576024.0
Iteration 52, inertia 92573896.0
Iteration 53, inertia 92571416.0
Iteration 54, inertia 92567920.0
Iteration 55, inertia 92563592.0
Iteration 56, inertia 92559064.0
Iteration 57, inertia 92554528.0
Iteration 58, inertia 92551664.0
Iteration 59, inertia 92550424.0
Iteration 60, inertia 92550096.0
Iteration 61, inertia 92549832.0
Iteration 62, inertia 92550032.0
Iteration 63, inertia 92550088.0
Iteration 

Iteration 9, inertia 92828056.0
Iteration 10, inertia 92775176.0
Iteration 11, inertia 92723440.0
Iteration 12, inertia 92667432.0
Iteration 13, inertia 92619840.0
Iteration 14, inertia 92582304.0
Iteration 15, inertia 92558088.0
Iteration 16, inertia 92545144.0
Iteration 17, inertia 92538248.0
Iteration 18, inertia 92534496.0
Iteration 19, inertia 92533112.0
Iteration 20, inertia 92531704.0
Iteration 21, inertia 92530872.0
Iteration 22, inertia 92529352.0
Iteration 23, inertia 92529400.0
Iteration 24, inertia 92529320.0
Iteration 25, inertia 92529056.0
Iteration 26, inertia 92528368.0
Iteration 27, inertia 92528112.0
Iteration 28, inertia 92527416.0
Iteration 29, inertia 92527184.0
Iteration 30, inertia 92527352.0
Iteration 31, inertia 92526952.0
Iteration 32, inertia 92527048.0
Iteration 33, inertia 92527256.0
Iteration 34, inertia 92527384.0
Iteration 35, inertia 92526760.0
Iteration 36, inertia 92526792.0
Iteration 37, inertia 92526664.0
Iteration 38, inertia 92526456.0
Iteration 3

Iteration 97, inertia 92547896.0
Iteration 98, inertia 92547952.0
Iteration 99, inertia 92547480.0
Iteration 100, inertia 92547616.0
Iteration 101, inertia 92547424.0
Iteration 102, inertia 92547112.0
Iteration 103, inertia 92547064.0
Iteration 104, inertia 92547208.0
Iteration 105, inertia 92547232.0
Iteration 106, inertia 92547512.0
Iteration 107, inertia 92547248.0
Iteration 108, inertia 92547136.0
Iteration 109, inertia 92546888.0
Iteration 110, inertia 92546952.0
Iteration 111, inertia 92546928.0
Iteration 112, inertia 92546912.0
Iteration 113, inertia 92546624.0
Iteration 114, inertia 92546472.0
Iteration 115, inertia 92545864.0
Iteration 116, inertia 92545576.0
Iteration 117, inertia 92544928.0
Iteration 118, inertia 92544888.0
Iteration 119, inertia 92544816.0
Iteration 120, inertia 92544856.0
Iteration 121, inertia 92544664.0
Iteration 122, inertia 92544464.0
Iteration 123, inertia 92544352.0
Iteration 124, inertia 92544464.0
Iteration 125, inertia 92544648.0
Iteration 126, in

In [193]:
'''
Encode Features
'''
train_encoded_features = []
index = 0

for features, shapes in zip(train_normalized_features, train_feature_shapes):
    encoded_features = codebook.encode(features, shapes)
    train_encoded_features.append(encoded_features)
    index += 1
    #print("Encoding Train Features ... {:4d}/{:4d}".format(index, len(train_normalized_features)))

print("train features encoded finished")

train features encoded finished


In [194]:
test_encoded_features = []
index = 0

for features, shapes in zip(test_normalized_features, test_feature_shapes):
    encoded_features = codebook.encode(features, shapes)
    test_encoded_features.append(encoded_features)
    index += 1
    #print("Encoding Text Features ... {:4d}/{:4d}".format(index, len(test_normalized_features)))
print("test feature encoded finished")

test feature encoded finished


[문제 3. SVM을 통한 이미지 분류 – 15 pts]  

3-1.   
Bag-of-Features 알고리즘을 통해 얻어진 인코딩된 벡터를 통해, SVM을 학습하시오.  
이 때, sklearn과 같은 패키지를 활용하여 SVM 학습을 구현하시면 됩니다. (자세한 사항은 템플릿 코드를 참고)

In [195]:
'''
Approximate Kernel
'''
chi2sampler = AdditiveChi2Sampler(sample_steps=2)
chi2sampler.fit(train_encoded_features, y_train)
train_encoded_features = chi2sampler.transform(train_encoded_features)
test_encoded_features = chi2sampler.transform(test_encoded_features)

In [196]:
'''
Classify Images with SVM
'''

start_time = time.time()

# 1) 아래의 model 부분에 sklearn 패키지를 활용하여, Linear SVM(SVC) 모델을 정의하세요.
#    처음에는 SVM의 parameter를 기본으로 설정하여 구동하시길 권장합니다.
#    구동 성공 시, SVM의 C 값과 max_iter 파라미터 등을 조정하여 성능 향상을 해보시길 바랍니다.
model = LinearSVC(C=700.0,
                  class_weight=None, dual=True,
                  fit_intercept=True,
                  intercept_scaling=1,
                  loss='squared_hinge',
                  max_iter= 100000, multi_class='ovr',
                  penalty='l2', random_state=0, tol= 1e-4,
                    verbose=0)

#model = LinearSVC(max_iter=10000) #svm의 param을 기본으로 설정하란게 뭔 말일까? 디폴트로 하란건가?

print("Classify Images ...")
model.fit(train_encoded_features, y_train)
train_score = model.score(train_encoded_features, y_train)
test_score = model.score(test_encoded_features, y_test)
elapsed_time = time.time() - start_time

Classify Images ...


C:\Users\qhrrl\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [197]:
'''
Print Results
'''
print()
print("=" * 90)
print("Train  Score: {:.5f}".format(train_score))
print("Test   Score: {:.5f}".format(test_score))
print("Elapsed Time: {:.2f} secs".format(elapsed_time))
print("=" * 90)


Train  Score: 0.15605
Test   Score: 0.14778
Elapsed Time: 54.50 secs


[문제 4. 파라미터 조정을 통한 성능 개선 – 40 pts] 

4-1.   
현재 주어진 코드에는 여러 하이퍼 파라미터가 존재합니다.    
예를 들어, Keypoint를 만들 때의 간격(patch_stride), codebook의 visual word 개수(K), SVM의 학습 파라미터 (C 값 및 max_iter) 등 이 있습니다.   
해당 파라미터들을 수정해가면서 결과를 개선시켜 보시오. (K와 patch_stride 같은 경우, 메모리 부족으로 큰 데이터를 사용하지 못 할 수 있으므로, 절대적인 성능 지표보다, 상대적인 성능 향상을 더 고려할 예정임)  

기존)  
=====================  
Train  Score: 0.17685  
Test   Score: 0.17230  
Elapsed Time: 4.69 secs  
=====================  
  
1차 ) patch_size = 32, SURF, K = 20, maxiter: 100,000(십만회)    
=> ORB를 시도해보려 했으나, PCA 과정에서 에러가 나서 잠시 보류, 오히려 점수가 떨어졌다 설마 loss score를 의미하는 것은 아닐테고...  
  
=====================  
Train  Score: 0.15605    
Test   Score: 0.14778  
Elapsed Time: 54.50 secs  
=====================  

4-2.  
앞서 여러 파라미터의 실험 결과를 토대로 개선된 혹은 개선 되지 않은 이유를 보고서에 설명하시오.   
(예를 들어, 각 파라미터 별로 결과 테이블 정리 및 결과에 대한 이유 등).

[심화 문제 5. 알고리즘 개선 – 가산점 +20 pts]  

(본 문제는 풀이를 하지 않으셔도 감점되지 않습니다.)  
5-1. Bag-of-Features 알고리즘의 단점 중 하나는 공간의 배열, 위치 관계를 보지 않고 특징을 인코딩한다는 점입니다.   
즉, 특정 특징들의 개수 만을 통해서 인코딩을 하기 때문에 물체의 위치, 배열 등의 패턴 차이는 제대로 표현되지 않을 수 있습니다.   
이러한 단점을 극복하기 위하여 Spatial Pyramid Matching (공간 분할)을 수행 후, codewords을 통해 인코딩 할 수 있습니다.   
앞서 언급된 Spatial Pyramid Matching 기법을 적용하여 성능을 개선 시켜 보시오.  
(https://slazebni.cs.illinois.edu/publications/pyramid_chapter.pdf를 참고하시면 도움이 됩니다.)